In [1]:
# If you are using Colab, run these commands
""" 
!wget -P dataset/ATIS https://raw.githubusercontent.com/BrownFortress/IntentSlotDatasets/main/ATIS/test.json
!wget -P dataset/ATIS https://raw.githubusercontent.com/BrownFortress/IntentSlotDatasets/main/ATIS/train.json
!wget https://raw.githubusercontent.com/BrownFortress/NLU-2024-Labs/main/labs/conll.py """

' \n!wget -P dataset/ATIS https://raw.githubusercontent.com/BrownFortress/IntentSlotDatasets/main/ATIS/test.json\n!wget -P dataset/ATIS https://raw.githubusercontent.com/BrownFortress/IntentSlotDatasets/main/ATIS/train.json\n!wget https://raw.githubusercontent.com/BrownFortress/NLU-2024-Labs/main/labs/conll.py '

In [2]:
# Global variables
import sys
import os
DEVICE = 'cuda:0'  # cuda:0 means we are using the GPU with id 0, if you have multiple GPU
EPOCHS = 200
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # Used to report errors on CUDA side
PAD_TOKEN = 0
sys.path.append('../src/')
from models import MultiModel1


In [3]:
import json
from pprint import pprint


def load_data(path):
    '''
        input: path/to/data
        output: json 
    '''
    dataset = []
    with open(path) as f:
        dataset = json.loads(f.read())
    return dataset


tmp_train_raw = load_data(os.path.join('dataset', 'ATIS', 'train.json'))
# we are producing the set to compare against
test_raw = load_data(os.path.join('dataset', 'ATIS', 'test.json'))
print('Train samples:', len(tmp_train_raw))
print('Test samples:', len(test_raw))

pprint(tmp_train_raw[0])

Train samples: 4978
Test samples: 893
{'intent': 'flight',
 'slots': 'O O O O O B-fromloc.city_name O B-depart_time.time '
          'I-depart_time.time O O O B-toloc.city_name O B-arrive_time.time O O '
          'B-arrive_time.period_of_day',
 'utterance': 'i want to fly from boston at 838 am and arrive in denver at '
              '1110 in the morning'}


In [4]:
import random
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter

# First we get the 10% of the training set, then we compute the percentage of these examples
# We are only splitting on intention but we should even split on labels of token

portion = 0.10

intents = [x['intent'] for x in tmp_train_raw]  # We stratify on intents
count_y = Counter(intents)

labels = []
inputs = []
mini_train = []

for id_y, y in enumerate(intents):
    if count_y[y] > 1:  # If some intents occurs only once, we put them in training
        inputs.append(tmp_train_raw[id_y])
        labels.append(y)
    else:
        mini_train.append(tmp_train_raw[id_y])
# Random Stratify
X_train, X_dev, y_train, y_dev = train_test_split(inputs, labels, test_size=portion,
                                                  random_state=42,
                                                  shuffle=True,
                                                  stratify=labels)
X_train.extend(mini_train)
train_raw = X_train
dev_raw = X_dev

y_test = [x['intent'] for x in test_raw]

# Intent distributions
print('Train:')
pprint({k: round(v/len(y_train), 3)*100 for k,
       v in sorted(Counter(y_train).items())})
print('Dev:'),
pprint({k: round(v/len(y_dev), 3)*100 for k,
       v in sorted(Counter(y_dev).items())})
print('Test:')
pprint({k: round(v/len(y_test), 3)*100 for k,
       v in sorted(Counter(y_test).items())})
print('='*89)
# Dataset size
print('TRAIN size:', len(train_raw))
print('DEV size:', len(dev_raw))
print('TEST size:', len(test_raw))

Train:
{'abbreviation': 2.9000000000000004,
 'aircraft': 1.6,
 'airfare': 8.5,
 'airline': 3.2,
 'airline+flight_no': 0.0,
 'airport': 0.4,
 'capacity': 0.3,
 'city': 0.4,
 'distance': 0.4,
 'flight': 73.7,
 'flight+airfare': 0.4,
 'flight_no': 0.2,
 'flight_time': 1.0999999999999999,
 'ground_fare': 0.4,
 'ground_service': 5.1,
 'meal': 0.1,
 'quantity': 1.0,
 'restriction': 0.1}
Dev:
{'abbreviation': 3.0,
 'aircraft': 1.6,
 'airfare': 8.4,
 'airline': 3.2,
 'airport': 0.4,
 'capacity': 0.4,
 'city': 0.4,
 'distance': 0.4,
 'flight': 73.7,
 'flight+airfare': 0.4,
 'flight_no': 0.2,
 'flight_time': 1.0,
 'ground_fare': 0.4,
 'ground_service': 5.0,
 'meal': 0.2,
 'quantity': 1.0,
 'restriction': 0.2}
Test:
{'abbreviation': 3.6999999999999997,
 'aircraft': 1.0,
 'airfare': 5.4,
 'airfare+flight': 0.1,
 'airline': 4.3,
 'airport': 2.0,
 'capacity': 2.4,
 'city': 0.7000000000000001,
 'day_name': 0.2,
 'distance': 1.0999999999999999,
 'flight': 70.8,
 'flight+airfare': 1.3,
 'flight+airline

In [5]:
from collections import Counter

w2id = {'pad': PAD_TOKEN, 'unk': 1}
slot2id = {'pad': PAD_TOKEN}
intent2id = {}
# Map the words only from the train set
# Map slot and intent labels of train, dev and test set. 'unk' is not needed.
for example in train_raw:
    for w in example['utterance'].split():
        if w not in w2id:
            w2id[w] = len(w2id)
    for slot in example['slots'].split():
        if slot not in slot2id:
            slot2id[slot] = len(slot2id)
    if example['intent'] not in intent2id:
        intent2id[example['intent']] = len(intent2id)

for example in dev_raw:
    for slot in example['slots'].split():
        if slot not in slot2id:
            slot2id[slot] = len(slot2id)
    if example['intent'] not in intent2id:
        intent2id[example['intent']] = len(intent2id)

for example in test_raw:
    for slot in example['slots'].split():
        if slot not in slot2id:
            slot2id[slot] = len(slot2id)
    if example['intent'] not in intent2id:
        intent2id[example['intent']] = len(intent2id)

print('# Vocab:', len(w2id)-2)  # we remove pad and unk from the count
print('# Slots:', len(slot2id)-1)
print('# Intent:', len(intent2id))

# Vocab: 864
# Slots: 129
# Intent: 26


In [6]:
from collections import Counter


class Lang():
    def __init__(self, words, intents, slots, cutoff=0):
        self.word2id = self.w2id(words, cutoff=cutoff, unk=True)
        self.slot2id = self.lab2id(slots)
        self.intent2id = self.lab2id(intents, pad=False)
        self.id2word = {v: k for k, v in self.word2id.items()}
        self.id2slot = {v: k for k, v in self.slot2id.items()}
        self.id2intent = {v: k for k, v in self.intent2id.items()}

    def w2id(self, elements, cutoff=None, unk=True):
        vocab = {'pad': PAD_TOKEN}
        if unk:
            vocab['unk'] = len(vocab)
        count = Counter(elements)
        for k, v in count.items():
            if v > cutoff:
                vocab[k] = len(vocab)
        return vocab

    def lab2id(self, elements, pad=True):
        vocab = {}
        if pad:
            vocab['pad'] = PAD_TOKEN
        for elem in elements:
            vocab[elem] = len(vocab)
        return vocab

In [7]:
words = sum([x['utterance'].split() for x in train_raw],
            [])  # No set() since we want to compute
# the cutoff
corpus = train_raw + dev_raw + test_raw  # We do not wat unk labels,
# however this depends on the research purpose
slots = set(sum([line['slots'].split() for line in corpus], []))
intents = set([line['intent'] for line in corpus])

lang = Lang(words, intents, slots, cutoff=0)

In [8]:
import torch
import torch.utils.data as data
# we are defining a data loader


class IntentsAndSlots (data.Dataset):
    # Mandatory methods are __init__, __len__ and __getitem__
    def __init__(self, dataset, lang, unk='unk'):
        self.utterances = []
        self.intents = []
        self.slots = []
        self.unk = unk

        for x in dataset:
            self.utterances.append(x['utterance'])
            self.slots.append(x['slots'])
            self.intents.append(x['intent'])

        self.utt_ids = self.mapping_seq(self.utterances, lang.word2id)
        self.slot_ids = self.mapping_seq(self.slots, lang.slot2id)
        self.intent_ids = self.mapping_lab(self.intents, lang.intent2id)

    def __len__(self):
        return len(self.utterances)

    def __getitem__(self, idx):  # it will be called for each element of the batch
        utt = torch.Tensor(self.utt_ids[idx])
        slots = torch.Tensor(self.slot_ids[idx])
        intent = self.intent_ids[idx]
        sample = {'utterance': utt, 'slots': slots, 'intent': intent}
        return sample

    # Auxiliary methods

    def mapping_lab(self, data, mapper):
        return [mapper[x] if x in mapper else mapper[self.unk] for x in data]

    def mapping_seq(self, data, mapper):  # Map sequences to number
        res = []
        for seq in data:
            tmp_seq = []
            for x in seq.split():
                if x in mapper:
                    tmp_seq.append(mapper[x])
                else:
                    tmp_seq.append(mapper[self.unk])
            res.append(tmp_seq)
        return res

In [9]:
from torch.utils.data import DataLoader


def collate_fn(data):
    def merge(sequences):
        '''
        merge from batch * sent_len to batch * max_len 
        '''
        lengths = [len(seq) for seq in sequences]
        max_len = 1 if max(lengths) == 0 else max(lengths)
        # Pad token is zero in our case
        # So we create a matrix full of PAD_TOKEN (i.e. 0) with the shape
        # batch_size X maximum length of a sequence
        padded_seqs = torch.LongTensor(
            len(sequences), max_len).fill_(PAD_TOKEN)
        for i, seq in enumerate(sequences):
            end = lengths[i]
            # We copy each sequence into the matrix so that the padding are the empty elements of the matrix
            padded_seqs[i, :end] = seq
        # print(padded_seqs)
        # We remove these tensors from the computational graph
        padded_seqs = padded_seqs.detach()
        return padded_seqs, lengths
    
    
    # Sort data by seq lengths
    data.sort(key=lambda x: len(x['utterance']), reverse=True)
    new_item = {}
    for key in data[0].keys():
        new_item[key] = [d[key] for d in data]

    # We just need one length for packed pad seq, since len(utt) == len(slots)
    src_utt, _ = merge(new_item['utterance'])
    y_slots, y_lengths = merge(new_item["slots"])
    intent = torch.LongTensor(new_item["intent"])

    src_utt = src_utt.to(DEVICE)  # We load the Tensor on our selected device
    y_slots = y_slots.to(DEVICE)
    intent = intent.to(DEVICE)
    y_lengths = torch.LongTensor(y_lengths).to(DEVICE)

    new_item["utterances"] = src_utt
    new_item["intents"] = intent
    new_item["y_slots"] = y_slots
    new_item["slots_len"] = y_lengths
    # the resulting matrix has dimension batch*max_token_sequence_of_the_batch
    return new_item




In [10]:
# Create our datasets
train_dataset_stock = IntentsAndSlots(train_raw, lang)
dev_dataset_stock = IntentsAndSlots(dev_raw, lang)
test_dataset_stock = IntentsAndSlots(test_raw, lang)


# Dataloader instantiations
train_loader_stock = DataLoader(train_dataset_stock, batch_size=128,
                                collate_fn=collate_fn,  shuffle=True)
dev_loader_stock = DataLoader(
    dev_dataset_stock, batch_size=64, collate_fn=collate_fn)
test_loader_stock = DataLoader(
    test_dataset_stock, batch_size=64, collate_fn=collate_fn)

In [46]:
import torch
import torch.utils.data as data
from transformers import BertTokenizer, BertModel

# we are defining a data loader


class IntentsAndSlots_BERTized (data.Dataset):
    # Mandatory methods are __init__, __len__ and __getitem__
    def __init__(self, dataset, unk='unk'):
        self.tokenizer = BertTokenizer.from_pretrained(
            "bert-base-uncased")
        self.utterances = []
        self.intents = []
        self.slots = []
        self.unk = unk

        for x in dataset:
            self.utterances.append(x['utterance'])
            self.slots.append(x['slots'])
            self.intents.append(x['intent'])
        
        self.intents_set = {elem for elem in self.intents}
        self.tokenizer.add_tokens(list(intents))

        self.intents_2_tok_id = {}
        for x in self.intents_set:
            self.intents_2_tok_id[x] = self.tokenizer.get_vocab()[x]

        self.int_id_2_tok_id = {}
        for id, x in enumerate(self.intents_set):
            self.int_id_2_tok_id[id] = self.tokenizer.get_vocab()[x]

        self.slots_set = []
        for elem in dataset:
            for slot in elem["slots"].split(" "):
                self.slots_set.append(slot)
        self.slots_set =set(self.slots_set)

        self.slot_tokenizer_2_id = {}
        for id,slot in enumerate(self.slots_set):
            self.slot_tokenizer_2_id[slot] = id
        
        self.slot_id_2_tok_id = {}
        for id, x in enumerate(self.slots_set):
            self.slot_id_2_tok_id[id] = self.slot_tokenizer_2_id[x]

        whole_utt = self.tokenizer(
            self.utterances, return_tensors="pt", padding=True)

        self.utt_ids = whole_utt["input_ids"].tolist()
        self.attention_masks = whole_utt["attention_mask"].tolist()
        self.token_type_ids = whole_utt["token_type_ids"].tolist()


        self.slot_ids = self.get_id_list(self.slots_set)
        self.intents = self.tokenizer(
            self.intents, return_tensors="pt", padding=True)["input_ids"].tolist()
        print(len(self.attention_masks))

    def __len__(self):
        return len(self.utterances)

    def __getitem__(self, idx):  # it will be called for each element of the batch
        utt = torch.Tensor(self.utt_ids[idx]).int().to(DEVICE)
        slots_tensor = #torch.Tensor(self.slot_ids[idx]).int().to(DEVICE)
        attention_mask = torch.Tensor(
            self.attention_masks[idx]).int().to(DEVICE)
        token_type_ids_tensor = # torch.Tensor(self.token_type_ids[idx]).int().to(DEVICE)
        intent_tensor = #torch.Tensor(self.intents[idx]).int().to(DEVICE)
        sample = {'utterance': utt, 'slots': slots,
                  'intents': get_intent_tensor(idx), "attention_mask": attention_mask, "token_type_ids": get_token_type_ids_tensor(idx)}

        return sample
    
    def get_id_list(self,obj,tokenizer):
        length = len(tokenizer)
        all_ids = []
        for slot_sequence in obj:
            sentence_ids = []
            for slot in slot_sequence:
                word_ids = []
                for i in range(length):
                    if i == tokenizer[slot]:
                        word_ids.append(1)
                    else:
                        word_ids.append(PAD_TOKEN)
                sentence_ids.append(word_ids)
            all_ids.append(sentence_ids)

    def get_slot_ids(self,id):
    

In [47]:
# Bert tokenized Dataset


train_dataset_bert = IntentsAndSlots_BERTized(train_raw)
dev_dataset_bert = IntentsAndSlots_BERTized(dev_raw)
test_dataset_bert = IntentsAndSlots_BERTized(test_raw)


# Dataloader instantiations
train_loader_bert = DataLoader(train_dataset_bert, batch_size=128,
                                 shuffle=True)
dev_loader_bert = DataLoader(
    dev_dataset_bert, batch_size=64)
test_loader_bert = DataLoader(
    test_dataset_bert, batch_size=64)

4480
498
893


In [48]:
sample = train_dataset_bert.__getitem__(0)
print(sample["intents"].size())
print(train_dataset_bert.__getrobo__(0))

torch.Size([3])
[101, 30522, 102]


In [14]:
import torch.nn as nn


def init_weights(mat):
    for m in mat.modules():
        if type(m) in [nn.GRU, nn.LSTM, nn.RNN]:
            for name, param in m.named_parameters():
                if 'weight_ih' in name:
                    for idx in range(4):
                        mul = param.shape[0]//4
                        torch.nn.init.xavier_uniform_(
                            param[idx*mul:(idx+1)*mul])
                elif 'weight_hh' in name:
                    for idx in range(4):
                        mul = param.shape[0]//4
                        torch.nn.init.orthogonal_(param[idx*mul:(idx+1)*mul])
                elif 'bias' in name:
                    param.data.fill_(0)
        else:
            if type(m) in [nn.Linear]:
                torch.nn.init.uniform_(m.weight, -0.01, 0.01)
                if m.bias != None:
                    m.bias.data.fill_(0.01)

In [15]:
from torch.utils.tensorboard import SummaryWriter


def log_values(writer, step, f1, acc, loss, prefix):
    if f1 != None:
        writer.add_scalar(f"{prefix}/f1", f1, step)
        writer.add_scalar(f"{prefix}/acc", acc, step)
    writer.add_scalar(f"{prefix}/loss", loss, step)

In [16]:
from conll import evaluate  # evaluates at chunk level
from sklearn.metrics import classification_report


def train_loop(data, optimizer, criterion_slots, criterion_intents, model, clip=5):
    model.train()
    loss_array = []
    for sample in data:
        optimizer.zero_grad()  # Zeroing the gradient
        slots, intent = model(
            utterance=sample['utterance'], attention_mask=sample['attention_mask'], token_type_ids=sample["token_type_ids"])
        print(intent.size(),sample["intents"].size())
        loss_intent = criterion_intents(intent, sample['intents'])
        loss_slot = criterion_slots(slots, sample['y_slots'])
        loss = loss_intent + loss_slot  # In joint training we sum the losses.
        # Is there another way to do that?
        # an example is to have an hyper parameter such that loss = alpha loss_intent + alpha-1 loss_slot
        loss_array.append(loss.item())
        loss.backward()  # Compute the gradient, deleting the computational graph
        # clip the gradient to avoid exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()  # Update the weights
    return loss_array


def eval_loop(data, criterion_slots, criterion_intents, model, lang):
    model.eval()
    loss_array = []

    ref_intents = []
    hyp_intents = []

    ref_slots = []
    hyp_slots = []
    # softmax = nn.Softmax(dim=1) # Use Softmax if you need the actual probability
    with torch.no_grad():  # It used to avoid the creation of computational graph
        for sample in data:
            slots, intents = model(
                sample['utterances'], sample['attention_mask'], sample["token_type_ids"])
            loss_intent = criterion_intents(intents, sample['intents'])
            loss_slot = criterion_slots(slots, sample['y_slots'])
            loss = loss_intent + loss_slot
            loss_array.append(loss.item())
            # Intent inference
            # Get the highest probable class
            out_intents = [lang.id2intent[x]
                           for x in torch.argmax(intents, dim=1).tolist()]
            gt_intents = [lang.id2intent[x]
                          for x in sample['intents'].tolist()]
            ref_intents.extend(gt_intents)
            hyp_intents.extend(out_intents)

            # Slot inference
            output_slots = torch.argmax(slots, dim=1)
            for id_seq, seq in enumerate(output_slots):
                length = sample['slots_len'].tolist()[id_seq]
                utt_ids = sample['utterance'][id_seq][:length].tolist()
                gt_ids = sample['y_slots'][id_seq].tolist()
                gt_slots = [lang.id2slot[elem] for elem in gt_ids[:length]]
                utterance = [lang.id2word[elem] for elem in utt_ids]
                to_decode = seq[:length].tolist()
                ref_slots.append([(utterance[id_el], elem)
                                for id_el, elem in enumerate(gt_slots)])
                tmp_seq = []
                for id_el, elem in enumerate(to_decode):
                    tmp_seq.append((utterance[id_el], lang.id2slot[elem]))
                hyp_slots.append(tmp_seq)
    try:
        results = evaluate(ref_slots, hyp_slots)
    except Exception as ex:
        # Sometimes the model predicts a class that is not in REF
        print("Warning:", ex)
        ref_s = set([x[1] for x in ref_slots])
        hyp_s = set([x[1] for x in hyp_slots])
        print(hyp_s.difference(ref_s))
        results = {"total": {"f": 0}}

    report_intent = classification_report(ref_intents, hyp_intents,
                                          zero_division=False, output_dict=True)
    return results, report_intent, loss_array

In [17]:
import matplotlib.pyplot as plt
from tqdm import tqdm
def train_model(param_string,exp_name,
                n_epochs, patience_set, model,optimizer,model_type):
    losses_train = []
    losses_dev = []
    sampled_epochs = []
    best_f1 = 0
    patience = patience_set
    writer = SummaryWriter(log_dir=f"runs/{param_string}")
    criterion_slots = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)
    criterion_intents = nn.CrossEntropyLoss()
    if model_type == "BERT":
        train_loader = train_loader_bert
        dev_loader = dev_loader_bert
        test_loader = test_loader_bert
    else: 
        train_loader = train_loader_stock
        dev_loader = dev_loader_stock
        test_loader = test_loader_stock

    for x in tqdm(range(1, n_epochs)):
        loss = train_loop(train_loader, optimizer, criterion_slots,
                          criterion_intents, model)
        if x % 5 == 0:  # We check the performance every 5 epochs
            sampled_epochs.append(x)
            losses_train.append(np.asarray(loss).mean())
            results_dev, intent_res, loss_dev = eval_loop(dev_loader, criterion_slots,
                                                          criterion_intents, model, lang)
            losses_dev.append(np.asarray(loss_dev).mean())

            f1 = results_dev['total']['f']
            log_values(writer, x,  f1,
                       intent_res['accuracy'], np.asarray(loss_dev).mean(), exp_name)
            # For decreasing the patience you can also use the average between slot f1 and intent accuracy
            if f1 > best_f1:
                best_f1 = f1
                # Here you should save the model
                patience = patience_set
            else:
                patience -= 1
            if patience <= 0:  # Early stopping with patience
                break  # Not nice but it keeps the code clean
        else:
            log_values(writer, x, None, None,
                       np.asarray(loss).mean(), exp_name)


    results_test, intent_test, _ = eval_loop(test_loader, criterion_slots,
                                             criterion_intents, model, lang)


    print('Slot F1: ', results_test['total']['f'])
    print('Intent Accuracy:', intent_test['accuracy'])
    return intent_test['accuracy'],model

In [18]:
import itertools
from models import Bertized

def train_grid_search(lr_list, hid_size_list, emb_size_list, weight_decay_list, patience_list, emb_drop_list, out_drop_list, vocab_len,out_slot,out_int, model_type="None",bidirectional=False, dropout=False, optimizer=None, n_epochs=100, exp_name="Pollo"):
    best_model = None
    best_accuracy = 0
    best_params = {}
    optimizer_class = optimizer
    model = None
    # Iterate over all combinations of the parameter lists
    for lr, hid_size, emb_size, weight_decay, patience, emb_drop in itertools.product(lr_list, hid_size_list, emb_size_list, weight_decay_list, patience_list, emb_drop_list):

        print(
            f"Training with parameters: lr={lr}, hid_size={hid_size}, emb_size={emb_size}, weight_decay={weight_decay}, patience={patience}, emb_drop={emb_drop}")

        if model_type == "LSTM":
        # Initialize the model with current parameters
            model = MultiModel1(emb_size=emb_size, hid_size=hid_size, vocab_len = vocab_len, model_type=model_type, out_slot=out_slot,
                            pad_index=PAD_TOKEN, dropout=dropout,out_int=out_int,
                                emb_dropout=emb_drop, bidirectional=bidirectional).to(DEVICE)
        else:
            model = Bertized(hid_size=hid_size, vocab_len=vocab_len, model_type=model_type, out_slot=out_slot,
                             pad_index=PAD_TOKEN, dropout=dropout, out_int=out_int,
                             emb_dropout=emb_drop).to(DEVICE)
        

        model.apply(init_weights)

        # Set up the optimizer with current parameters
        optimizer_obj = optimizer_class(model.parameters(), lr=lr,
                                        weight_decay=weight_decay)

        # Train the model
        accuracy, model = train_model(exp_name+"/"+f"LR={lr}, HS={hid_size}, ES={emb_size}, WD={weight_decay}, P={patience}, E_D={emb_drop},M_T={model_type},D={dropout}", model=model,
                                      optimizer=optimizer_obj,patience_set=patience,
                                      n_epochs=n_epochs, exp_name=exp_name,model_type = model_type)

        # Check if the current model is the best so far
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_params = {
                'lr': lr,
                'hid_size': hid_size,
                'emb_size': emb_size,
                'weight_decay': weight_decay,
                'patience': patience,
                'emb_drop': emb_drop,
            }

            torch.save({
                'model_state_dict': model.state_dict(),
                "best_params":best_params,
                "exp":exp_name
            }, "models/"+exp_name+".model")

    print(f"Best model parameters: {best_params}")
    print(f"Best model accuracy: {best_accuracy}")
    return best_model, best_params

In [19]:
from torch import optim

out_slot = len(lang.slot2id)
out_int = len(lang.intent2id)
vocab_len = len(lang.word2id)
print(out_slot, out_int, vocab_len)

130 26 866


In [20]:
lr_list = [.001, .0005, .0001]
hid_size_list = [200, 300, 500]
emb_size_list = [200, 300, 500]
weight_decay_list = [0.0001, 0.001]
patience_list = [3]
emb_drop_list = [.01, .1, .15]
out_drop_list = [0]

best_model, best_params = train_grid_search(
    lr_list, hid_size_list, emb_size_list,
    weight_decay_list, patience_list, emb_drop_list,
    out_drop_list, vocab_len=vocab_len, out_slot=out_slot, out_int=out_int, exp_name="BERT_STOCK", model_type="BERT", optimizer=optim.Adam, n_epochs=EPOCHS)

Training with parameters: lr=0.001, hid_size=200, emb_size=200, weight_decay=0.0001, patience=3, emb_drop=0.01


  0%|          | 0/199 [00:00<?, ?it/s]

torch.Size([128, 51, 26]) torch.Size([128, 196])


RuntimeError: Expected target size [128, 26], got [128, 196]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs --host localhost --port 8088